# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [1]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.ml import feature
from pyspark.ml import regression
from pyspark.sql import functions as fn
from pyspark.sql import Row
from pyspark import sql

## Warning: Use exclusively Spark. Do not use Pandas at all in this assignment

# Part 2: Dataframes and Spark ML

In this section, you will learn to create dataframes from messy data and then perform simple regression on it.

There is some mysterious process generating data, stored in `/datasets/host_server_requests`, with the following format:

`feature1|feature2|...|featurem => outcome`

`feature1` can be either "HOST" or "SERVER" and from feature $2$ through $m$ are floating point numbers.

In [4]:
requests_rdd = sc.textFile('/datasets/host_server_requests')

## Question 1:

In this question, you will create a function `process_line` that receives a line from `/datasets/host_server_requests` and returns a `Row` object with the following columns: 

- You will codify the first feature as a column `f1` with a `1` if the source is `HOST` and `0` otherwise
- You will create 7 other features that you assign to columns `f2`, `f3`, ..., through `f8`
- Finally, you will assign the outcome to the column `label`
- Remember to make all features of type `float`.

For the following code:


```python
requests_rdd.map(process_line).take(10)
```

it should generate the following:

```python
[Row(f1=1.0, f2=2e-05, f3=0.80279, f4=-0.09174, f5=0.04041, f6=-0.22504, f7=-0.0504, f8=0.58149, label=163.877101489),
 Row(f1=1.0, f2=5e-05, f3=-0.00454, f4=-0.0211, f5=0.00174, f6=-0.11684, f7=0.19182, f8=-0.23745, label=-105.023368852),
 Row(f1=1.0, f2=0.00015, f3=-0.10437, f4=0.04869, f5=0.18333, f6=-0.21864, f7=0.27638, f8=-0.13441, label=-115.011801582),
 Row(f1=1.0, f2=-0.00015, f3=0.27118, f4=0.14526, f5=0.06101, f6=0.13401, f7=0.06237, f8=-0.74065, label=-122.623452696),
 Row(f1=1.0, f2=-6e-05, f3=0.1413, f4=0.12084, f5=0.05452, f6=0.09272, f7=0.2534, f8=-0.65331, label=-117.130523174),
 Row(f1=1.0, f2=-8e-05, f3=-0.41534, f4=-0.04205, f5=-0.00724, f6=-0.07463, f7=0.13273, f8=0.19112, label=-73.5775668047),
 Row(f1=1.0, f2=-8e-05, f3=-0.45937, f4=-0.23509, f5=-0.05679, f6=0.06077, f7=-0.49597, f8=-0.30668, label=-137.37933148),
 Row(f1=0.0, f2=2e-05, f3=-0.23465, f4=0.07345, f5=-0.07217, f6=-0.19256, f7=-0.14377, f8=-0.15183, label=-162.804738349),
 Row(f1=0.0, f2=-7e-05, f3=-0.10321, f4=0.27467, f5=0.04058, f6=-0.24541, f7=0.08631, f8=-0.2979, label=-212.111291232),
 Row(f1=1.0, f2=-7e-05, f3=-0.01039, f4=-0.00453, f5=-0.01352, f6=-0.05199, f7=-0.3772, f8=-0.19641, label=-91.5022329392)]
```


In [19]:
# YOUR CODE HERE

#raise NotImplementedError()

[['SERVER'], ['0.00008'], ['0.45592'], ['0.25362'], ['-0.12878'], ['-0.06930'], ['-0.14740'], ['-0.53736', '-106.947151761']]


In [12]:
#requests_rdd.take(1)[0]
def process_line(st):
    a=st.split("|")
    row_dir={}
    i=0
    while i<len(a):
        k="f"+str(i+1)
        if i==0:
            if a[i]=="HOST":
                row_dir[k]=1.0
            else:
                row_dir[k]=0.0
        else:
            if "=>" in a[i]:
                row_dir[k]=float(a[i].split(" =>  ")[0])
                row_dir["label"]=float(a[i].split(" =>  ")[1])
            else:
                row_dir[k]=float(a[i])
        i=i+1
    return Row(**row_dir)

In [13]:
# try it here
requests_rdd.map(process_line).take(10)

[Row(f1=0.0, f2=8e-05, f3=0.45592, f4=0.25362, f5=-0.12878, f6=-0.0693, f7=-0.1474, f8=-0.53736, label=-106.947151761),
 Row(f1=0.0, f2=-1e-05, f3=0.10405, f4=-0.17047, f5=0.10127, f6=0.13776, f7=0.66619, f8=0.32221, label=127.061566761),
 Row(f1=1.0, f2=-1e-05, f3=-0.72394, f4=0.31161, f5=0.06975, f6=0.0086, f7=-0.44817, f8=-0.22976, label=-170.222653879),
 Row(f1=1.0, f2=0.00024, f3=-0.12887, f4=-0.01287, f5=0.18377, f6=-0.07004, f7=0.10632, f8=-0.40884, label=-127.850350049),
 Row(f1=1.0, f2=6e-05, f3=-0.06767, f4=0.08402, f5=0.14438, f6=-0.19704, f7=0.2839, f8=-0.12487, label=-110.295233798),
 Row(f1=1.0, f2=5e-05, f3=0.59406, f4=0.25195, f5=-0.00541, f6=-0.05218, f7=0.12202, f8=-0.1285, label=27.7039261796),
 Row(f1=0.0, f2=-8e-05, f3=-0.08562, f4=-0.00398, f5=-0.17124, f6=-0.13839, f7=-0.23696, f8=-0.814, label=-301.722572842),
 Row(f1=1.0, f2=-0.0001, f3=-0.28606, f4=-0.00447, f5=-0.10446, f6=-0.25006, f7=-0.03288, f8=-0.87445, label=-361.98704195),
 Row(f1=0.0, f2=-0.00018, f3=

In [14]:
# 5 pts
np.testing.assert_equal(len(requests_rdd.map(process_line).first()), 9)
np.testing.assert_equal(requests_rdd.map(process_line).count(), 10000)

## Question 2:

Transform the `requests_rdd` RDD into a Spark 2.0 DataFrame and store it in `requests_df`

In [15]:
# YOUR CODE HERE
requests_df=requests_rdd.map(process_line).toDF()
#raise NotImplementedError()

In [16]:
# 5 pts
np.testing.assert_equal(type(requests_df), sql.dataframe.DataFrame)
np.testing.assert_equal(set(requests_df.columns), {'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'label'})
np.testing.assert_equal(requests_df.count(), 10000)

## Question 3:

In this question, we will explore the data. We have a hypothesis that depending on whether the request was from the "HOST" or "SERVER" (`f1` column), there are significant difference in the outcome (`label` column).

You will find whether this is true by computing two quantities for each group of `f1`. You will compute the mean outcome and the *standard error of the mean* or SE of the outcome. The equation for SE of a variable $x$ is:

$$\text{SE}(x) = \frac{\text{std}(x)}{\sqrt{n}}$$

From `requests_df`, create a dataframe `summary_df` that contains, for each value of `f1`, the mean `label` as a column `mlabel` and the SEM of `label` as a column `semlabel`. For the SE equation, use the *sample standard devivation* computed by `fn.stddev_samp`. **Hint: perform an aggregate operation and use appropriate combinations of functions in the package `fn`. Rename columns appropriately**

In [54]:
# YOUR CODE HERE
#Making a dataframe for the mean values for 0.0 and 1.0:
mlbs=requests_df.groupBy("f1").agg({"label":"mean"})
mlbs=mlbs.withColumnRenamed("avg(label)","mlabel")

#Making a dataframe for the standard deviation for 0.0 and 1.0:
st=requests_df.groupBy("f1").agg({"label":"stddev_samp"})

#Number of instances for each category:
nlbs=requests_df.groupBy("f1").agg({"label":"count"})

#Joining standard deviation and number of instances into a single dataframe and calcuating standard error in another column:
sem=nlbs.join(st,"f1")
sem=sem.withColumn("semlabel",st["stddev_samp(label)"]/fn.sqrt(nlbs["count(label)"]))

#Final join between mean and remaining values and dropping count and standard deviation:
mlbs=mlbs.join(sem,"f1")
summary_df=mlbs.drop("count(label)","stddev_samp(label)")
summary_df.show(2)
#raise NotImplementedError()

+---+-------------------+------------------+
| f1|             mlabel|          semlabel|
+---+-------------------+------------------+
|0.0|-29.611753412328927|1.7554606758419875|
|1.0| -12.62243193686321|1.7481077347771363|
+---+-------------------+------------------+



The schema of `summary_df` should look like:

```python
summary_df.printSchema()
```
```console
root
 |-- f1: double (nullable = true)
 |-- mlabel: double (nullable = true)
 |-- semlabel: double (nullable = true)

```
The mean label for each `f1` feature should be:

```python
summary_df.select('f1', 'mlabel').show()
```

```console
+---+------------------+
| f1|            mlabel|
+---+------------------+
|0.0|-29.61175341232892|
|1.0|-12.62243193686321|
+---+------------------+
```

In [55]:
# 10 pts
np.testing.assert_equal(summary_df.count(), 2)
np.testing.assert_equal(set(summary_df.columns), {'f1', 'mlabel', 'semlabel'})
np.testing.assert_approx_equal(summary_df.rdd.map(lambda r: r.mlabel).sum(), -42.23418534919213,
                              significant=3)
np.testing.assert_approx_equal(summary_df.rdd.map(lambda r: r.semlabel).sum(), 3.503568410619124,
                              significant=3)

Statistically, we say that a mean $\overline{x}$ is *statistically* different from another mean $\overline{y}$ if it is not within 2 standard errors of the mean of x (i.e., $\overline{y} \not\in \{\overline{x} \pm 2\text{SE}(x)\}$). Discuss whether you think the mean label for f1 = 0 is statistically different from the mean label for f1 = 1

In [56]:
# display it to make your argument
summary_df.show()

+---+-------------------+------------------+
| f1|             mlabel|          semlabel|
+---+-------------------+------------------+
|0.0|-29.611753412328927|1.7554606758419875|
|1.0| -12.62243193686321|1.7481077347771363|
+---+-------------------+------------------+



**3 pts for your answer below:**

In [60]:
mean_list=summary_df.select("mlabel").collect()
se_list=summary_df.select("semlabel").collect()
m_0=mean_list[0].mlabel   #Mean for 0.0
m_1=mean_list[1].mlabel    #Mean for 1.0
se_0=se_list[0].semlabel   #Standard error for 0.0
se_1=se_list[1].semlabel   #Standard error for 1.0

#Calcualting values at 2 standard deviations on both sides from the mean for 0.0:
lower_lim_0_sd_2=m_0-2*se_0
upper_lim_0_sd_2=m_0+2*se_0

#Checking if the mean for 1.0 lies between the 2 standard deviations of the mean for 0.0 on either sides: 
if (m_1>=lower_lim_0_sd_2 and m_1<=upper_lim_0_sd_2):
    print("The 2 means are not statistically different.")
else:
    print("The 2 means are statiscally different.")

The 2 means are statiscally different.


The 2 means are statistically different based on the code above

## Question 4:

Use the transformer `VectorAssembler` to create a dataframe that puts all columns `f1`, `f2`, ..., `f8` from `requests_df` into a column named `features`. Assign the vector assembler object into a variable `va` and the new dataframe into the variable  `features_df`

In [68]:
# YOUR CODE HERE
cols=[]
for col in requests_df.columns:
    if "f" in col:
        cols.append(col)
va=feature.VectorAssembler(inputCols=cols,outputCol="features")
features_df=va.transform(requests_df)
#raise NotImplementedError()

The schema of the new dataframe should be like this:

```python
features_df.printSchema()
```

```console
root
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- f3: double (nullable = true)
 |-- f4: double (nullable = true)
 |-- f5: double (nullable = true)
 |-- f6: double (nullable = true)
 |-- f7: double (nullable = true)
 |-- f8: double (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
```

In [69]:
# try it here
features_df.show(3)

+---+-------+--------+--------+--------+-------+--------+--------+--------------+--------------------+
| f1|     f2|      f3|      f4|      f5|     f6|      f7|      f8|         label|            features|
+---+-------+--------+--------+--------+-------+--------+--------+--------------+--------------------+
|0.0| 8.0E-5| 0.45592| 0.25362|-0.12878|-0.0693| -0.1474|-0.53736|-106.947151761|[0.0,8.0E-5,0.455...|
|0.0|-1.0E-5| 0.10405|-0.17047| 0.10127|0.13776| 0.66619| 0.32221| 127.061566761|[0.0,-1.0E-5,0.10...|
|1.0|-1.0E-5|-0.72394| 0.31161| 0.06975| 0.0086|-0.44817|-0.22976|-170.222653879|[1.0,-1.0E-5,-0.7...|
+---+-------+--------+--------+--------+-------+--------+--------+--------------+--------------------+
only showing top 3 rows



In [70]:
# 5 pts
np.testing.assert_equal(type(features_df), sql.dataframe.DataFrame)
np.testing.assert_equal(set(features_df.columns), 
                        {'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'features', 'label'})

## Question 5:

Run a linear regression model on `features_df` using the `features` column to predict the `label` column. Store the transformer fit to the data in the `lr_model` variable (the transformer is what the estimator's `fit` function returns). Use the transformer to create a dataframe named `predictions_df` with two columns: `label` and `prediction` based on the `features_df` dataframe.

In [79]:
# YOUR CODE HERE
lr=regression.LinearRegression(featuresCol="features",labelCol="label",maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model=lr.fit(features_df)
print("Coefficients: "+str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

predictions_df=lr_model.transform(features_df)
predictions_df=predictions_df.select("label","prediction")
predictions_df.show(5)
#raise NotImplementedError()

Coefficients: [17.625919361532464,166414.78271509538,141.90187235799473,0.0,0.0,358.677012519037,0.0,228.07655342226417]
Intercept: -28.78029963178382
+--------------+-------------------+
|         label|         prediction|
+--------------+-------------------+
|-106.947151761| -98.18674908367636|
| 127.061566761| 107.22033388272486|
|-170.222653879|-164.86521617888468|
|-127.850350049|-107.87028276739508|
|-110.295233798| -109.9256307824003|
+--------------+-------------------+
only showing top 5 rows



The resulting dataframe should be in `predictions_df`. Running `predictions_df.show(5)` should produce something like

```python
predictions_df.show(5)
```

```console
+--------------+-------------------+
|         label|         prediction|
+--------------+-------------------+
| 163.877101489| 159.06994708337518|
|-105.023368852| -99.52598722329135|
|-115.011801582|-109.91382979074436|
|-122.623452696|-118.62864861627764|
|-117.130523174|-116.89245751669506|
+--------------+-------------------+
only showing top 5 rows
```


In [78]:
# 5 pts:
np.testing.assert_equal(set(predictions_df.columns), {'label', 'prediction'})
np.testing.assert_equal(predictions_df.count(), 10000)
np.testing.assert_equal(type(lr_model), regression.LinearRegressionModel)
np.testing.assert_equal(type(va), feature.VectorAssembler)

The root mean squared error is defined as

$$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2}$$

Combine functions in `fn` package and other functions to create a dataframe called `rmse_df` that contains the root mean squared error in column `rmse` based on the `predictions_df` dataframe.

In [91]:
# YOUR CODE HERE
#Creating a dataframe with the residual error terms:
rms=predictions_df.withColumn("e",(predictions_df["prediction"]-predictions_df["label"]))

#Adding a column for the square of the residual terms:
rms=rms.withColumn("sq_e",(rms["e"]**2))

#Creating a new dataframe and aggregating to add all the squared residual terms:
rmse_df=rms.groupBy().agg({"sq_e":"sum"})

#Calculating the total number of elements:
n=rms.groupBy().agg({"e":"count"}).collect()[0][0]

#Dividing the aggregated squared residual terms and dividing them with the total number of terms and then square root(RMSE):
rmse_df=rmse_df.withColumn("rmse",fn.sqrt((1/n * rmse_df["sum(sq_e)"])))

#Finally, dropping unwanted columns:
rmse_df=rmse_df.drop("sum(sq_e)")
rmse_df.show(1)

#raise NotImplementedError()

+-----------------+
|             rmse|
+-----------------+
|8.538945505198058|
+-----------------+



In [92]:
# 5 pts
np.testing.assert_array_less(rmse_df.first().rmse, 10)
np.testing.assert_equal(rmse_df.count(), 1)

## Question 6

Add the constant 10 to each feature in `requests_df` and redo the analytics pipeline from Q5, creating a vector assembler `va2`, `features2_df`, `lr_model2`, and `predictions2_df`. When adding the constant 10, preserve the column names in `features2_df` such that the columns in this new dataframe should be `f1` thorugh `f8`. Do not modify the column `label`. Fit a new linear model, similar to question 4, compute its root mean square error, and store it in `rmse2_df`.

In [99]:
# YOUR CODE HERE

requests_df2=requests_df.select(requests_df['f1']+10,requests_df['f2']+10,requests_df['f3']+10,requests_df['f4']+10,requests_df['f5']+10,requests_df['f6']+10,requests_df['f7']+10,requests_df['f8']+10,requests_df["label"])
cols2=["f1","f2","f3","f4","f5","f6","f7","f8","label"]
rq2=requests_df2.toDF(*cols2)

cols_2=[]
for col2 in rq2.columns:
    if "f" in col2:
        cols_2.append(col2)
va2=feature.VectorAssembler(inputCols=cols_2,outputCol="features")
features2_df=va2.transform(rq2)

lr2=regression.LinearRegression(featuresCol="features",labelCol="label",maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model2=lr2.fit(features2_df)
print("Coefficients: "+str(lr_model2.coefficients))
print("Intercept: " + str(lr_model2.intercept))

predictions2_df=lr_model2.transform(features2_df)
predictions2_df=predictions2_df.select("label","prediction")
predictions2_df.show(5)

rms2=predictions2_df.withColumn("e",(predictions2_df["prediction"]-predictions2_df["label"]))
rms2=rms2.withColumn("sq_e",(rms2["e"]**2))
rmse2_df=rms2.groupBy().agg({"sq_e":"sum"})
n2=rms2.groupBy().agg({"e":"count"}).collect()[0][0]
rmse2_df=rmse2_df.withColumn("rmse",fn.sqrt((1/n * rmse2_df["sum(sq_e)"])))
rmse2_df=rmse2_df.drop("sum(sq_e)")
rmse2_df.show(1)

#raise NotImplementedError()

Coefficients: [17.625913988313656,166411.82306045984,141.9018882088414,0.0,0.0,358.67701606012173,0.0,228.07654583016975]
Intercept: -1671609.8245451555
+--------------+-------------------+
|         label|         prediction|
+--------------+-------------------+
|-106.947151761| -98.18697484582663|
| 127.061566761| 107.22036311915144|
|-170.222653879| -164.8652017065324|
|-127.850350049|-107.87099769548513|
|-110.295233798|-109.92581460834481|
+--------------+-------------------+
only showing top 5 rows

+-----------------+
|             rmse|
+-----------------+
|8.538955501612891|
+-----------------+



In [100]:
# 10 pts:
np.testing.assert_equal(set(predictions2_df.columns), {'label', 'prediction'})
np.testing.assert_equal(predictions2_df.count(), 10000)
np.testing.assert_equal(type(va2), feature.VectorAssembler)
np.testing.assert_equal(type(lr_model2), regression.LinearRegressionModel)
np.testing.assert_array_less(rmse2_df.first().rmse, 10)
np.testing.assert_equal(rmse2_df.count(), 1)

Compare the root mean squared error of Q5 and Q6. They should be almost exactly the same. Also, compare the coefficients and intercepts of `lr_model` and `lr_model2`. 

In [101]:
print("RMSE Q5")
rmse_df.show()
print("RMSE Q6")
rmse2_df.show()
print("intercept lr_model", lr_model.intercept)
print("intercept lr_model2", lr_model2.intercept)
print("coefficients lr_model", lr_model.coefficients)
print("coefficients lr_model2", lr_model2.coefficients)

RMSE Q5
+-----------------+
|             rmse|
+-----------------+
|8.538945505198058|
+-----------------+

RMSE Q6
+-----------------+
|             rmse|
+-----------------+
|8.538955501612891|
+-----------------+

intercept lr_model -28.78029963178382
intercept lr_model2 -1671609.8245451555
coefficients lr_model [17.625919361532464,166414.78271509538,141.90187235799473,0.0,0.0,358.677012519037,0.0,228.07655342226417]
coefficients lr_model2 [17.625913988313656,166411.82306045984,141.9018882088414,0.0,0.0,358.67701606012173,0.0,228.07654583016975]


**2 pts for your answer below:** Comment on why there are differences or similarities between coefficients, intercepts, RMSE, even though we modified the features. You can use [Latex in Markdown](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html#LaTeX-equations) to put some math into your explanation.

**Q5:**

**Coefficients: [17.625919361532464,166414.78271509538,141.90187235799473,0.0,0.0,358.677012519037,0.0,228.07655342226417]**

**Intercept: -28.78029963178382**

**RMSE: 8.538945505198058**


**Q6:**

**Coefficients: [17.625913988313656,166411.82306045984,141.9018882088414,0.0,0.0,358.67701606012173,0.0,228.07654583016975]**

**Intercept: -1671609.8245451555**

**RMSE: 8.538955501612891**

The change in the coefficients isn't that big; the minute change is the result of addition of the noise (+10) to every column. The intercept, however, has changed drastically (went from -28.78029963178382 to -1671609.8245451555). This change is the reflection of the the noise term getting multiplied by the coefficients and aggregating to the increase in slope. RMSE has a very minute change because the models fit almost identically, as the residual error terms won't change much.